**Классификация изображений с помощью сверточных нейронных сетей**

В данном задании Вам необходимо разработать архитектуру сверточной ИНС, обеспечивающую наибольшую точность при ограничении на количество операций (FLOPs <= 0.707e6).
Заготовка кода для выполнения задания приведена выше. Вашей задачей будет заполнить пропущеные места, которые отмечены ключевым словом *None*.
Необходимая точность (accuracy) сети на датасете CIFAR100 - 30%
Желаемая точность (accuracy) сети на датасете CIFAR100 - 45%

In [1]:
!pip install keras-flops

In [2]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd
import tensorflow as tf
from keras_flops import get_flops

In [3]:
# Глобальные константы
CLASSES       = 100
BATCH_SIZE    = 128
LEARNING_RATE = 1e-2

In [4]:
from keras.datasets import cifar100

In [5]:
# Выполните загрузку модели

(X_train, y_train), (X_val, y_val) = cifar100.load_data()

In [6]:
# Преобразуйте метки классов в one_hot формат
y_train = tf.keras.utils.to_categorical(y_train, num_classes=CLASSES)
y_val = tf.keras.utils.to_categorical(y_val, num_classes=CLASSES)

In [7]:
# убедитесь, что данная ячейка выполняется без ошибок
assert X_train.shape == (50000, 32, 32, 3)
assert X_val.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 100)
assert y_val.shape == (10000, 100)

In [110]:
# Задайте архитектуру модели
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=[32,32,3]),
    tf.keras.layers.Conv2D(16, 4, strides = 2, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(4, strides=2, padding='same'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(CLASSES),
    tf.keras.layers.Activation('softmax')
])

In [111]:
# вычисление количества операций
flops = get_flops(model, batch_size=1)
print(f"FLOPs: {(flops / 1e6):.4f}e6")

FLOPs: 0.6274e6


In [112]:
# вывод краткой информации о модели
model.summary()

Model: "sequential_31"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_31 (Conv2D)          (None, 16, 16, 16)        784       
                                                                 
 batch_normalization_31 (Bat  (None, 16, 16, 16)       64        
 chNormalization)                                                
                                                                 
 activation_62 (Activation)  (None, 16, 16, 16)        0         
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 8, 8, 16)         0         
 g2D)                                                            
                                                                 
 flatten_31 (Flatten)        (None, 1024)              0         
                                                                 
 dense_31 (Dense)            (None, 100)             

In [113]:
# параметры данной ячейки могут быть изменены для получения более высокой точности
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [115]:
# обучение модели
model.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_val, y_val),
    batch_size=BATCH_SIZE,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(filepath="{epoch:02d}-{val_accuracy:.2f}.hdf5", save_best_only=True),
        
    ],
    use_multiprocessing=True,
    workers=8,
    epochs=12
)

Epoch 1/12
391/391 [==============================] - 8s 21ms/step - loss: 2.3616 - accuracy: 0.4141 - val_loss: 2.8176 - val_accuracy: 0.3273
Epoch 2/12
391/391 [==============================] - 8s 21ms/step - loss: 2.3289 - accuracy: 0.4221 - val_loss: 2.8616 - val_accuracy: 0.3251
Epoch 3/12
391/391 [==============================] - 8s 20ms/step - loss: 2.3036 - accuracy: 0.4289 - val_loss: 2.8427 - val_accuracy: 0.3235
Epoch 4/12
391/391 [==============================] - 9s 22ms/step - loss: 2.2751 - accuracy: 0.4330 - val_loss: 2.9543 - val_accuracy: 0.3142
Epoch 5/12
391/391 [==============================] - 8s 22ms/step - loss: 2.2465 - accuracy: 0.4377 - val_loss: 2.8229 - val_accuracy: 0.3367
Epoch 6/12
391/391 [==============================] - 8s 21ms/step - loss: 2.2254 - accuracy: 0.4453 - val_loss: 2.7900 - val_accuracy: 0.3433
Epoch 7/12
391/391 [==============================] - 8s 21ms/step - loss: 2.2036 - accuracy: 0.4480 - val_loss: 2.8969 - val_accuracy: 0.3255

In [116]:
scores = model.evaluate(X_val, y_val, verbose=0) 
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 34.34%
